In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import requests
import dotenv
import os
import json
from bs4 import BeautifulSoup


In [2]:
ct = contrans()
ct.mypassword
congresskey= ct.congresskey

In [3]:
useragent = ct.get_useragent()
useragent


'python-requests/2.32.3'

In [4]:
headers=ct.make_headers()
headers

{'User-Agent': 'python-requests/2.32.3', 'From': 'mathaimadelyn@gmail.com'}

In [5]:
print(ct.get_bioguideIDs)


<bound method contrans.get_bioguideIDs of <contrans.contrans object at 0x1370e0500>>


In [6]:
goodbioguide = ct.get_bioguide(name = 'good', state = 'Virginia', district = 5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."
1,G000280,5.0,"goode, virgil h., jr.",Republican,Virginia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/G000280?for...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/g000280_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [7]:
goodbioguide['bioguideId'].reset_index()

,index,bioguideId
0,0,G000595
1,1,G000280


In [8]:
goodid = goodbioguide['bioguideId'].reset_index()['bioguideId'][0]

In [9]:
goodid

'G000595'

In [10]:
#Sponsored legislation
goodsbills = ct.get_sponsoredlegislation(goodid)


In [11]:
r = requests.get(goodsbills[0]['url'],
                 params = {'api_key':congresskey}) #goodsbills[0]['url']

texturl = json.loads(r.text)['bill']['textVersions']['url']


r = requests.get(texturl,
                 params= {'api_key':congresskey})
json.loads(r.text)['textVersions'][0]['formats'][0]['url']

toscrape = json.loads(r.text)['textVersions'][0]['formats'][0]['url']

In [12]:
from bs4 import BeautifulSoup

r = requests.get(toscrape)
mysoup = BeautifulSoup(r.text, 'html.parser') #equivalent of json.loads, allows you to search through a string
billtext = mysoup.text

In [13]:
billurl = goodsbills[0]['url']

In [14]:
ct.get_billdata(billurl)

NameError: name 'BeautifulSoup' is not defined

In [ ]:
print(mysoup.text)

In [ ]:
votes = ct.get_votes()
votes

In [ ]:
house = house(['rollnumber', 'icpsr','cast_code'])

In [ ]:
house_mat = pd.merge(house, house on = 'rollnumber')
house_mat

In [ ]:
house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree': 'mean'})
house_mat = house_mat.reset_index()
house_mat

In [ ]:
ideo = ct.get_ideology()
members = ideo[['bioname', 'icpsr','partycode']]
members

In [ ]:
agreement = pd.merge(house_mat, members, 
                     left_on = 'icpsr_x', 
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member1',
                              'party_code':'party1'}, axis=1)
agreement = agreement.drop(['icspr_x'],axis=1)
agreement = pd.merge(agreement, members,
                     left_on = 'icspr_y',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member2',
                              'party_code':'party2'}, axis=1)
agreement = agreement[['member1','party1','member2','party2','agree']]
agreement

In [ ]:
agreement[agreement['member1'].str.contains('GOOD,')].sort_values('agree', ascending = False).head(30)